In [1]:
import random
# coding=utf-8

# The proven prime
Pcurve = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1  

# Number of points in the field
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

# This defines the curve. y^2 = x^3 + Acurve * x + Bcurve
Acurve = 0
Bcurve = 7  

Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
GPoint = (Gx, Gy)  

# replace with any private key
privKey = 2125
RandNum = random.randrange(1, N-1)
# RandNum = 28695618543805844332113829720373285210420739438570883203839696518176414791234
# the hash of your message/transaction
HashOfThingToSign = 86032112319101611046176971828093669637772856272773459297323797145286374828050


def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)


def modinv(a, m):# Extended Euclidean Algorithm/'division' in elliptic curves
    if a < 0:
        a += m
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m


# Not true addition, invented for EC. It adds Point-P with Point-Q.
def ECadd(x1, y1, x2, y2):
    LamNumer = y2 - y1
    LamDenom = x2 - x1
    s = (LamNumer * modinv(LamDenom, Pcurve)) % Pcurve
    x3 = (s * s - x1 - x2) % Pcurve
    y3 = (s * (x1 - x3) - y1) % Pcurve
    return (x3, y3)


# EC point doubling, invented for EC. It doubles Point-P.
def ECdouble(x1, y1):
    LamNumer = 3 * x1 ** 2 + Acurve
    LamDenom = 2 * y1
    s = (LamNumer * modinv(LamDenom, Pcurve)) % Pcurve
    x3 = (s * s - 2 * x1) % Pcurve
    y3 = (s * (x1 - x3) - y1) % Pcurve
    return (x3, y3)


def EccMultiply(xs, ys, Scalar):  # Double & add. EC Multiplication, Not true multiplication
    if Scalar == 0 or Scalar >= N:
        raise Exception("Invalid Scalar/Private Key")

    ScalarBin = str(bin(Scalar))[2:]
  
    Qx, Qy = xs, ys
    for i in range(1, len(ScalarBin)):  # This is invented EC multiplication.
        Qx, Qy = ECdouble(Qx, Qy)  # print "DUB", Qx; print
        # print("DUB")
        if ScalarBin[i] == "1":
            # print ("ADD")
            Qx, Qy = ECadd(Qx, Qy, xs, ys)  # print "ADD", Qx; print
            
    return (Qx, Qy)


print
print ("******* Public Key Generation *********")
xPublicKey, yPublicKey = EccMultiply(Gx, Gy, privKey)
print ("the private key (in base 10 format):")
print (privKey)
print
print ("the public key:")
print ("xPublicKey:", xPublicKey)
print ("yPublicKey:", yPublicKey)
print

print ("******* Signature Generation *********")
xRandSignPoint, yRandSignPoint = EccMultiply(Gx, Gy, RandNum)
r = xRandSignPoint % N
print ("r =", r)
s = ((HashOfThingToSign + r * privKey) * (modinv(RandNum, N))) % N
print ("s =", s)

print
print ("******* Signature Verification *********")
w = modinv(s, N)
xu1, yu1 = EccMultiply(Gx, Gy, (HashOfThingToSign * w) % N)
xu2, yu2 = EccMultiply(xPublicKey, yPublicKey, (r * w) % N)
x, y = ECadd(xu1, yu1, xu2, yu2)
if (r % N  == x % N):
    print("Verification Success")
else: 
    print("Verification Fail")


******* Public Key Generation *********
the private key (in base 10 format):
2125
the public key:
xPublicKey: 101781878184007084671381261094362501380942865028961237641824038035511380961002
yPublicKey: 58369962163175720309519279668836655893250110774156566655796446087224164166873
******* Signature Generation *********
r = 102575602159522900328953093897017399147538835379946168053673485310685299135750
s = 87513453838023375510744589333164280874190211655294144182234991991364913867780
******* Signature Verification *********
Verification Success
